In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
sonetos = np.load('sonetosSigloXIX.npy')

In [3]:
sonetos.shape

(2695,)

In [4]:
from models import replace_chars
input_text = []
target_text = []
for soneto in sonetos:
    soneto = replace_chars(soneto.lower())
    input_text.append('<sos> ' + soneto)
    target_text.append(soneto + ' <eos>')

Using TensorFlow backend.


In [5]:
input_text[2]

'<sos> y miras a jesús char_comma  virgen maría char_exclamation_close  char_new_line y latiendo tu pecho de quebranto char_new_line a mares viertes congojoso llanto char_new_line y aun brama de furor la turna impía char_exclamation_close  char_new_line  char_new_line  char_exclamation_open y goza contemplando su agonía char_comma  char_new_line y no se abate de mortal espanto char_exclamation_close  char_dot  char_dot  char_dot  char_new_line  char_exclamation_open y al ver la pena en tu semblante santo char_new_line su alma a la piedad se ostenta fría char_exclamation_close  char_new_line  char_new_line  char_question_open mas char_comma  quién char_comma   char_exclamation_open maría char_exclamation_close  brindará consuelo char_new_line a la honda angustia que en tu ser impera char_comma  char_new_line si ella cubre a la vez de luto el cielo char_question_close  char_new_line  char_new_line  char_exclamation_open y es tan ardiente y tan profunda y fiera char_comma  char_new_line q

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [7]:
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(input_text + target_text)

In [8]:
input_sonetos = tokenizer.texts_to_sequences(input_text)
target_sonetos = tokenizer.texts_to_sequences(target_text)

In [9]:
len(tokenizer.word_index)

24679

In [10]:
tokenizer.word_index['<eos>']

28

In [11]:
print(input_sonetos[0][:10])
print(target_sonetos[0][:10])

[27, 12122, 9, 51, 386, 277, 12123, 2, 1, 69]
[12122, 9, 51, 386, 277, 12123, 2, 1, 69, 38]


In [12]:
print(input_sonetos[0][-10:])
print(target_sonetos[0][-10:])

[25, 3827, 8, 22, 3374, 4, 2283, 3, 1, 1]
[3827, 8, 22, 3374, 4, 2283, 3, 1, 1, 28]


In [13]:
len(input_sonetos[0]), len(target_sonetos[0])

(138, 138)

In [14]:
MAX_LEN = max([len(soneto) for soneto in input_sonetos])

In [15]:
VOCAB_SIZE = len(tokenizer.word_index)

In [16]:
input_sonetos_padded = pad_sequences(input_sonetos, maxlen=MAX_LEN, padding='post')
target_sonetos_padded = pad_sequences(target_sonetos, maxlen=MAX_LEN, padding='post')

In [17]:
print(input_sonetos_padded[0][:150])
print(target_sonetos_padded[0][:150])

[   27 12122     9    51   386   277 12123     2     1    69    38  1009
    48  1196     2     5 12124     3     1   629    26  2280     2   585
     2     8   102    63  1814     2     1     9    23   175     9   686
     4   122  8226     3     1     1    35  4396     9     4    51   305
  8227     2     1     9     7   205  2719     5  2461    69  8228     1
    44     2   606 12125     2 12126    36     1    19    94  2281     9
    41  2282     2    25  8229    18     1     1   540    10   460     3
 12127     2   585   109     3     1    80  2280    23    26     2 12128
     2    32   607     3     1   276   111     2    94  8230   386    47
   282  5236     3     1     1   777     2  5237     2   257   156  5238
     2     1    44 12129 12130     2 12131     1    25  3827     8    22
  3374     4  2283     3     1     1     0     0     0     0     0     0
     0     0     0     0     0     0]
[12122     9    51   386   277 12123     2     1    69    38  1009    48
  1196     2 

In [18]:
from keras.utils import to_categorical

In [19]:
target_sonetos_padded.shape

(2695, 187)

In [20]:
NUM_SONETOS = 500

In [21]:
target_sonetos_padded_cat = to_categorical(target_sonetos_padded[:NUM_SONETOS]-1, num_classes=VOCAB_SIZE)

In [22]:
target_sonetos_padded_cat.shape

(500, 187, 24679)

In [23]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [24]:
word_dim = 64
lstm_units = 128
model = Sequential()
model.add(Embedding(VOCAB_SIZE+1, word_dim, mask_zero = True))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(Dense(VOCAB_SIZE, activation="softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          1579520   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         98816     
_________________________________________________________________
dense_1 (Dense)              (None, None, 24679)       3183591   
Total params: 4,861,927
Trainable params: 4,861,927
Non-trainable params: 0
_________________________________________________________________


In [25]:
from fnn_helper import PlotLosses
batch_size = 32
plot_losses = PlotLosses(plot_interval=1, evaluate_interval=None)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(input_sonetos_padded[:NUM_SONETOS], 
          target_sonetos_padded_cat, 
          batch_size=batch_size, epochs=2, verbose = 1, callbacks=[plot_losses])

<Figure size 2000x500 with 2 Axes>